In [1]:
'''
  Reference : https://github.com/roomylee/cnn-relation-extraction
'''

import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

## 파라미터 설정

In [2]:
class Config:
    # Data loading params
    max_sentence_length = 90
    dev_sample_percentage = 0.1
    
    # Embeddings
    embedding_path = ''
    text_embedding_dim = 300
    pos_embedding_dim = 50
    
    # CNN
    filter_sizes = '2,3,4,5'
    num_filters = 128
    dropout_keep_prob = 0.5
    l2_reg_lambda = 1e-5
    
    # Training parameters
    batch_size = 20
    num_epochs = 100
    display_every = 50
    evaluate_every = 100
    num_checkpoints = 5
    learning_rate = 1.0
    decay_rate = 0.9
    
    # Testing parameters
    checkpoint_dir = ''

    labels_count = 19
    class2label = {'Other': 0,
               'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2,
               'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4,
               'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6,
               'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8,
               'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10,
               'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12,
               'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14,
               'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16,
               'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}

    label2class = {0: 'Other',
                   1: 'Message-Topic(e1,e2)', 2: 'Message-Topic(e2,e1)',
                   3: 'Product-Producer(e1,e2)', 4: 'Product-Producer(e2,e1)',
                   5: 'Instrument-Agency(e1,e2)', 6: 'Instrument-Agency(e2,e1)',
                   7: 'Entity-Destination(e1,e2)', 8: 'Entity-Destination(e2,e1)',
                   9: 'Cause-Effect(e1,e2)', 10: 'Cause-Effect(e2,e1)',
                   11: 'Component-Whole(e1,e2)', 12: 'Component-Whole(e2,e1)',
                   13: 'Entity-Origin(e1,e2)', 14: 'Entity-Origin(e2,e1)',
                   15: 'Member-Collection(e1,e2)', 16: 'Member-Collection(e2,e1)',
                   17: 'Content-Container(e1,e2)', 18: 'Content-Container(e2,e1)'}

## Dataset 

Load Relation Extraction dataset of SemEval2010 task8

In [3]:
import nltk
import re
import os

class Dataset:
    def clean_str(self, text):
        text = text.lower()
        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"that's", "that is ", text)
        text = re.sub(r"there's", "there is ", text)
        text = re.sub(r"it's", "it is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

        return text.strip()

    def load_data_and_labels(self, path):
        # Data Format
        # 1\t"The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>."
        # Component-Whole(e2,e1)
        # Comment: Not a collection: there is structure here, organisation.
        # 
        # 2\t"The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord."
        # Other
        # Comment:
        # 
        data = []
        lines = [line.strip() for line in open(path)]
        max_sentence_length = 0
        for idx in range(0, len(lines), 4):
            id = lines[idx].split("\t")[0]
            
            # Sentence
            sentence = lines[idx].split("\t")[1][1:-1]
            sentence = sentence.replace('<e1>', ' _e11_ ')
            sentence = sentence.replace('</e1>', ' _e12_ ')
            sentence = sentence.replace('<e2>', ' _e21_ ')
            sentence = sentence.replace('</e2>', ' _e22_ ')

            sentence = self.clean_str(sentence)
            tokens = nltk.word_tokenize(sentence)
            sentence = " ".join(tokens)
            
            # Max Sentence Length
            if max_sentence_length < len(tokens):
                max_sentence_length = len(tokens)
                
            # e1, e2 position
            e1 = tokens.index("e12") - 1
            e2 = tokens.index("e22") - 1
            
            # Relative Position
            pos1 = ""
            pos2 = ""
            for word_idx in range(len(tokens)):
                pos1 += str((Config.max_sentence_length - 1) + word_idx - e1) + " "
                pos2 += str((Config.max_sentence_length - 1) + word_idx - e2) + " "
                
            # Label
            relation = lines[idx + 1]
            label = Config.class2label[relation]
            data.append([id, sentence, e1, e2, pos1, pos2, relation, label])

        print(path)
        print("max sentence length = {}\n".format(max_sentence_length))

        df = pd.DataFrame(data=data, columns=["id", "sentence", "e1", "e2", 'pos1', 'pos2', 'relation', 'label'])
        return df
    
    def download_and_load_datasets(self):
        dataset = tf.keras.utils.get_file(
          fname="SemEval2010_task8_all_data.zip", 
          origin="https://s3.ap-northeast-2.amazonaws.com/bowbowbow-storage/dataset/SemEval2010_task8_all_data.zip", 
          extract=True)
        
        train_file = 'SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT'
        test_file = 'SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT'
        
        train_df = self.load_data_and_labels(os.path.join(os.path.dirname(dataset), train_file))
        test_df = self.load_data_and_labels(os.path.join(os.path.dirname(dataset), test_file))
        return train_df, test_df

dataset = Dataset()
train_df, test_df = dataset.download_and_load_datasets()
train_df.head()

/home/seungwon/.keras/datasets/SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT
max sentence length = 89

/home/seungwon/.keras/datasets/SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT
max sentence length = 68



,id,sentence,e1,e2,pos1,pos2,relation,label
0,1,the system as described above has its greatest...,13,18,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 8...,"Component-Whole(e2,e1)",12
1,2,the e11 child e12 was carefully wrapped and bo...,2,12,87 88 89 90 91 92 93 94 95 96 97 98 99 100 101...,77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 9...,Other,0
2,3,the e11 author e12 of a keygen uses a e21 disa...,2,10,87 88 89 90 91 92 93 94 95 96 97 98 99 100 101...,79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 9...,"Instrument-Agency(e2,e1)",6
3,4,a misty e11 ridge e12 uprises from the e21 sur...,3,9,86 87 88 89 90 91 92 93 94 95 96,80 81 82 83 84 85 86 87 88 89 90,Other,0
4,5,the e11 student e12 e21 association e22 is the...,2,5,87 88 89 90 91 92 93 94 95 96 97 98 99 100 101...,84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 9...,"Member-Collection(e1,e2)",15


## Word2Vec 함수 정의

In [4]:
def load_word2vec(embedding_dim, vocab):
    download_path = tf.keras.utils.get_file(
      fname="GoogleNews-vectors-negative300.bin.gz", 
      origin="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", 
      extract=True)

    embedding_path = os.path.join(os.path.dirname(download_path), 'GoogleNews-vectors-negative300.bin')
    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) / np.sqrt(len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load word2vec file {0}".format(embedding_path))
    with open(embedding_path, "rb") as f:
        header = f.readline()
        vocab_size, layer_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            idx = vocab.vocabulary_.get(word)
            if idx != 0:
                initW[idx] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return initW

## CNN Model

In [5]:
class TextCNN:
    def __init__(self, 
               sequence_length, 
               num_classes, 
               text_vocab_size, 
               text_embedding_size, 
               pos_vocab_size, 
               pos_embedding_size, 
               filter_sizes,
               num_filters, 
               l2_reg_lambda=0.0):
        
        self.input_text = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_text')
        self.input_p1 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p1')
        self.input_p2 = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_p2')
        self.input_y = tf.placeholder(tf.float32, shape=[None, num_classes], name='input_y')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        
        # Embedding layer
        with tf.variable_scope('text-embedding'):
            self.W_text = tf.Variable(tf.random_uniform([text_vocab_size, text_embedding_size], -0.25, 0.25), name='W_text')
            self.text_embedded_chars = tf.nn.embedding_lookup(self.W_text, self.input_text)
            self.text_embedded_chars_expanded = tf.expand_dims(self.text_embedded_chars, -1)
            
        with tf.variable_scope('position-embedding'):
            self.W_pos = tf.get_variable('W_pos', [pos_vocab_size, pos_embedding_size], initializer=tf.keras.initializers.glorot_normal())
            self.p1_embedded_chars = tf.nn.embedding_lookup(self.W_pos, self.input_p1)
            self.p2_embedded_chars = tf.nn.embedding_lookup(self.W_pos, self.input_p2)
            self.p1_embedded_chars_expanded = tf.expand_dims(self.p1_embedded_chars, -1)
            self.p2_embedded_chars_expanded = tf.expand_dims(self.p2_embedded_chars, -1)
        
        self.embedded_chars_expanded = tf.concat([self.text_embedded_chars_expanded,
                                                  self.p1_embedded_chars_expanded,
                                                  self.p2_embedded_chars_expanded], 2)
        
        _embedding_size = text_embedding_size + 2*pos_embedding_size
        
        # convolution + maxpool layer
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope('conv-maxpool-{}'.format(filter_size)):
                conv = tf.layers.conv2d(self.embedded_chars_expanded, 
                                       num_filters, 
                                       [filter_size, _embedding_size], 
                                       kernel_initializer=tf.keras.initializers.glorot_normal(),
                                       activation=tf.nn.relu,
                                       name='conv')
                pooled = tf.nn.max_pool(conv, 
                                        ksize=[1, sequence_length - filter_size + 1, 1, 1],
                                        strides=[1, 1, 1, 1],
                                        padding='VALID',
                                        name='pool'
                                       )
                pooled_outputs.append(pooled)
                
        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        
        self.h_pool = tf.concat(pooled_outputs, 3) # [batch_size, 1, 1, num_filters]
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        # Add dropout
        with tf.variable_scope('dropout'):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
        
        # Final scores and predictions
        with tf.variable_scope('output'):
            self.logits = tf.layers.dense(self.h_drop, num_classes, kernel_initializer=tf.keras.initializers.glorot_normal())
            self.predictions = tf.argmax(self.logits, 1, name='predictions')
        
        # Calculate mean corss-entropy loss
        with tf.variable_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.input_y)
            self.l2 = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables()])
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * self.l2
        
        # Accuracy    
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')

## Preprocessing

In [6]:
# Build vocabulary
text_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(Config.max_sentence_length)
x = np.array(list(text_vocab_processor.fit_transform(train_df['sentence'])))
y = np.array([np.eye(Config.labels_count)[label] for label in train_df['label']]) # One-hot encoding 

print('Text Vocabulary Size {}'.format(len(text_vocab_processor.vocabulary_)))
print('X = {}'.format(x.shape))
print('Y = {}'.format(y.shape))

# Example: pos1[3] = '95 96 97 98 99 100 101 999 999 999 ... 999' (without offset: [-2 -1  0  1  2   3   4 999 999 999 ... 999])
# =>
# [11 12 13 14 15  16  21  17  17  17 ...  17]
pos_vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(Config.max_sentence_length)
pos_vocab_processor.fit(train_df['pos1'] + train_df['pos2'])

p1 = np.array(list(pos_vocab_processor.transform(train_df['pos1'])))
p2 = np.array(list(pos_vocab_processor.transform(train_df['pos2'])))

print('Position Vocabulary Size {}'.format(len(pos_vocab_processor.vocabulary_)))
print('position1 = {}'.format(p1.shape))
print('position2 = {}'.format(p2.shape))

# Randomly shuffle data to split into train and dev
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled, p1_shuffled, p2_shuffled, y_shuffled = x[shuffle_indices], p1[shuffle_indices], p2[shuffle_indices], y[shuffle_indices]

# Split train/dev set
dev_sample_index = -1 * int(Config.dev_sample_percentage*float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:] 
p1_train, p1_dev = p1_shuffled[:dev_sample_index], p1_shuffled[dev_sample_index:] 
p2_train, p2_dev = p2_shuffled[:dev_sample_index], p2_shuffled[dev_sample_index:] 
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:] 
print("Train/Dev split: {:d}/{:d}\n".format(len(y_train), len(y_dev)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Text Vocabulary Size 19151
X = (8000, 90)
Y = (8000, 19)
Position Vocabulary Size 162
position1 = (8000, 90)
position2 = (8000, 90)
Train/Dev split: 7200/800



## Function for train

In [7]:
# Pre-trained word2vec
def load_word2vec(embedding_dim, vocab):
    download_path = tf.keras.utils.get_file(
      fname="GoogleNews-vectors-negative300.bin.gz", 
      origin="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", 
      extract=True)
    
    embedding_path = os.path.join(os.path.dirname(download_path), 'GoogleNews-vectors-negative300.bin')
    if not os.path.exists(embedding_path):
        print('unzip :', embedding_path)
        import gzip
        import shutil
        with gzip.open('{}.gz'.format(embedding_path), 'rb') as f_in:
            with open(embedding_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(vocab.vocabulary_), embedding_dim).astype(np.float32) / np.sqrt(len(vocab.vocabulary_))
    # load any vectors from the word2vec
    print("Load word2vec file {0}".format(embedding_path))
    with open(embedding_path, "rb") as f:
        header = f.readline()
        vocab_size, layer_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            idx = vocab.vocabulary_.get(word)
            if idx != 0:
                initW[idx] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return initW

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

## Train

In [8]:
import datetime
import time

from sklearn.metrics import f1_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

sess = tf.Session()
with sess.as_default():
    cnn = TextCNN(
        sequence_length=x_train.shape[1],
        num_classes=y_train.shape[1],
        text_vocab_size=len(text_vocab_processor.vocabulary_),
        text_embedding_size=Config.text_embedding_dim,
        pos_vocab_size=len(pos_vocab_processor.vocabulary_),
        pos_embedding_size=Config.pos_embedding_dim,
        filter_sizes=list(map(int, Config.filter_sizes.split(','))),
        num_filters=Config.num_filters,
        l2_reg_lambda=Config.l2_reg_lambda
    )
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    optimizer = tf.train.AdadeltaOptimizer(Config.learning_rate, Config.decay_rate, 1e-6)
    gvs = optimizer.compute_gradients(cnn.loss)
    capped_gvs = [(tf.clip_by_value(grad, -1.0, 1.0), var) for grad, var in gvs]
    train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)
    
    # Output directory for models and summary
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "23.runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", cnn.loss)
    acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
    
    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, acc_summary])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Dev summaries
    dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
    dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
    dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

    # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
    checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
    checkpoint_prefix = os.path.join(checkpoint_dir, "model")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=Config.num_checkpoints)
    
    sess.run(tf.global_variables_initializer())

    pretrain_W = load_word2vec(Config.text_embedding_dim, text_vocab_processor)
    sess.run(cnn.W_text.assign(pretrain_W))
    print("Success to load pre-trained word2vec model!\n")
    
    # Generate batches
    batches = batch_iter(list(zip(x_train, p1_train, p2_train, y_train)),
                                      Config.batch_size, 
                                      Config.num_epochs)
    
    # Training loop. For each batch...
    best_f1 = 0.0  # For save checkpoint(model)
    for batch in batches:
        x_batch, p1_batch, p2_batch, y_batch = zip(*batch)
        # Train
        feed_dict = {
            cnn.input_text: x_batch,
            cnn.input_p1: p1_batch,
            cnn.input_p2: p2_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: Config.dropout_keep_prob
        }
        _, step, summaries, loss, accuracy = sess.run(
            [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy], feed_dict)
        train_summary_writer.add_summary(summaries, step)

        # Training log display
        if step % Config.display_every == 0:
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

        # Evaluation
        if step % Config.evaluate_every == 0:
            print("\nEvaluation:")
            feed_dict = {
                cnn.input_text: x_dev,
                cnn.input_p1: p1_dev,
                cnn.input_p2: p2_dev,
                cnn.input_y: y_dev,
                cnn.dropout_keep_prob: 1.0
            }
            summaries, loss, accuracy, predictions = sess.run(
                [dev_summary_op, cnn.loss, cnn.accuracy, cnn.predictions], feed_dict)
            dev_summary_writer.add_summary(summaries, step)

            time_str = datetime.datetime.now().isoformat()
            f1 = f1_score(np.argmax(y_dev, axis=1), predictions, labels=np.array(range(1, 19)), average="macro")
            print("{}: step {}, loss {:g}, acc {:.4f}".format(time_str, step, loss, accuracy))
            print("[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): {:g}\n".format(f1))

            # Model checkpoint
            if best_f1 < f1:
                best_f1 = f1
                path = saver.save(sess, checkpoint_prefix + "-{:.3g}".format(best_f1), global_step=step)
                print("Saved model checkpoint to {}\n".format(path))



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Writing to /home/seungwon/project/tf-notes/23.runs/1556799363

embedding_path : /home/seungwon/.keras/datasets/GoogleNews-vectors-negative300.bin
Load word2vec file /home/seungwon/.keras/datasets/GoogleNews-vectors-negative300.bin


/home/seungwon/project/tf-notes/venv/lib/python3.5/site-packages/ipykernel_launcher.py:38: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Success to load pre-trained word2vec model!

2019-05-02T21:16:22.061238: step 10, loss 3.5824, acc 0.1
2019-05-02T21:16:22.204289: step 20, loss 3.28312, acc 0.25
2019-05-02T21:16:22.349888: step 30, loss 3.08921, acc 0.3
2019-05-02T21:16:22.495009: step 40, loss 2.95683, acc 0.5
2019-05-02T21:16:22.639890: step 50, loss 3.15307, acc 0.4
2019-05-02T21:16:22.784836: step 60, loss 2.75135, acc 0.4
2019-05-02T21:16:22.930432: step 70, loss 2.68975, acc 0.45
2019-05-02T21:16:23.075216: step 80, loss 2.2193, acc 0.6
2019-05-02T21:16:23.219182: step 90, loss 2.27854, acc 0.6
2019-05-02T21:16:23.364240: step 100, loss 2.28184, acc 0.5

Evaluation:
2019-05-02T21:16:23.863139: step 100, loss 2.32106, acc 0.495
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.408042

Saved model checkpoint to /home/seungwon/project/tf-notes/23.runs/1556799363/checkpoints/model-0.408-100

2019-05-02T21:16:24.174095: step 110, loss 2.32465, acc 0.45
2019-05-02T21:16:24.318039: step 120, loss 2.

2019-05-02T21:16:37.884968: step 960, loss 0.977397, acc 1
2019-05-02T21:16:38.030611: step 970, loss 1.31821, acc 0.8
2019-05-02T21:16:38.176247: step 980, loss 1.12757, acc 0.95
2019-05-02T21:16:38.318574: step 990, loss 1.21412, acc 0.9
2019-05-02T21:16:38.463942: step 1000, loss 1.38291, acc 0.75

Evaluation:
2019-05-02T21:16:38.540112: step 1000, loss 1.64266, acc 0.74375
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.716093

2019-05-02T21:16:38.688627: step 1010, loss 1.12798, acc 0.95
2019-05-02T21:16:38.831969: step 1020, loss 1.44536, acc 0.8
2019-05-02T21:16:38.975616: step 1030, loss 1.41141, acc 0.8
2019-05-02T21:16:39.121112: step 1040, loss 1.40804, acc 0.8
2019-05-02T21:16:39.267136: step 1050, loss 1.41897, acc 0.85
2019-05-02T21:16:39.412061: step 1060, loss 1.32259, acc 0.8
2019-05-02T21:16:39.557917: step 1070, loss 1.21148, acc 0.9
2019-05-02T21:16:39.702067: step 1080, loss 1.24257, acc 0.85
2019-05-02T21:16:39.850008: step 1090, loss 1.22254,

2019-05-02T21:16:54.193282: step 2000, loss 0.997847, acc 0.95

Evaluation:
2019-05-02T21:16:54.268645: step 2000, loss 1.67898, acc 0.76375
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.744054

2019-05-02T21:16:54.414410: step 2010, loss 0.862622, acc 1
2019-05-02T21:16:54.558612: step 2020, loss 0.957138, acc 0.95
2019-05-02T21:16:54.704568: step 2030, loss 0.910998, acc 0.95
2019-05-02T21:16:54.851129: step 2040, loss 0.926069, acc 0.95
2019-05-02T21:16:54.998077: step 2050, loss 0.984803, acc 0.95
2019-05-02T21:16:55.144239: step 2060, loss 1.01386, acc 0.9
2019-05-02T21:16:55.290136: step 2070, loss 0.863986, acc 1
2019-05-02T21:16:55.434286: step 2080, loss 0.891004, acc 1
2019-05-02T21:16:55.580678: step 2090, loss 1.08984, acc 0.9
2019-05-02T21:16:55.726955: step 2100, loss 1.00275, acc 0.95

Evaluation:
2019-05-02T21:16:55.803509: step 2100, loss 1.60298, acc 0.78875
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.771323

2019-05-02T

...

2019-05-02T21:24:25.998869: step 31220, loss 0.479565, acc 1
2019-05-02T21:24:26.145751: step 31230, loss 0.480873, acc 1
2019-05-02T21:24:26.291073: step 31240, loss 0.479836, acc 1
2019-05-02T21:24:26.437948: step 31250, loss 0.481642, acc 1
2019-05-02T21:24:26.583694: step 31260, loss 0.481153, acc 1
2019-05-02T21:24:26.730609: step 31270, loss 0.47869, acc 1
2019-05-02T21:24:26.876709: step 31280, loss 0.478834, acc 1
2019-05-02T21:24:27.022980: step 31290, loss 0.478664, acc 1
2019-05-02T21:24:27.169207: step 31300, loss 0.478622, acc 1

Evaluation:
2019-05-02T21:24:27.245935: step 31300, loss 1.46004, acc 0.8
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.785212

2019-05-02T21:24:27.392387: step 31310, loss 0.47901, acc 1
2019-05-02T21:24:27.537217: step 31320, loss 0.479203, acc 1
2019-05-02T21:24:27.684015: step 31330, loss 0.478274, acc 1
2019-05-02T21:24:27.830869: step 31340, loss 0.478204, acc 1
2019-05-02T21:24:27.976126: step 31350, loss 0.478432, ac

2019-05-02T21:24:42.859636: step 32310, loss 0.469237, acc 1
2019-05-02T21:24:43.006471: step 32320, loss 0.469879, acc 1
2019-05-02T21:24:43.152527: step 32330, loss 0.469432, acc 1
2019-05-02T21:24:43.299139: step 32340, loss 0.469594, acc 1
2019-05-02T21:24:43.445538: step 32350, loss 0.469105, acc 1
2019-05-02T21:24:43.593299: step 32360, loss 0.468799, acc 1
2019-05-02T21:24:43.739257: step 32370, loss 0.469187, acc 1
2019-05-02T21:24:43.886239: step 32380, loss 0.468987, acc 1
2019-05-02T21:24:44.033108: step 32390, loss 0.468621, acc 1
2019-05-02T21:24:44.180082: step 32400, loss 0.468892, acc 1

Evaluation:
2019-05-02T21:24:44.256344: step 32400, loss 1.44156, acc 0.79875
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.787863

2019-05-02T21:24:44.405752: step 32410, loss 0.469347, acc 1
2019-05-02T21:24:44.551715: step 32420, loss 0.472378, acc 1
2019-05-02T21:24:44.698791: step 32430, loss 0.469648, acc 1
2019-05-02T21:24:44.844303: step 32440, loss 0.4716

2019-05-02T21:24:59.731755: step 33400, loss 1.44626, acc 0.79375
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.7862

2019-05-02T21:24:59.879530: step 33410, loss 0.459051, acc 1
2019-05-02T21:25:00.024886: step 33420, loss 0.459117, acc 1
2019-05-02T21:25:00.172973: step 33430, loss 0.459977, acc 1
2019-05-02T21:25:00.321607: step 33440, loss 0.459878, acc 1
2019-05-02T21:25:00.469791: step 33450, loss 0.458992, acc 1
2019-05-02T21:25:00.616952: step 33460, loss 0.459474, acc 1
2019-05-02T21:25:00.764875: step 33470, loss 0.459636, acc 1
2019-05-02T21:25:00.913314: step 33480, loss 0.459444, acc 1
2019-05-02T21:25:01.063369: step 33490, loss 0.459178, acc 1
2019-05-02T21:25:01.211328: step 33500, loss 0.467026, acc 1

Evaluation:
2019-05-02T21:25:01.288570: step 33500, loss 1.4349, acc 0.79625
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.784013

2019-05-02T21:25:01.436701: step 33510, loss 0.458176, acc 1
2019-05-02T21:25:01.583326: step 

2019-05-02T21:25:16.390843: step 34480, loss 0.450306, acc 1
2019-05-02T21:25:16.537879: step 34490, loss 0.452054, acc 1
2019-05-02T21:25:16.684643: step 34500, loss 0.449749, acc 1

Evaluation:
2019-05-02T21:25:16.761792: step 34500, loss 1.44234, acc 0.7925
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.780107

2019-05-02T21:25:16.907807: step 34510, loss 0.449735, acc 1
2019-05-02T21:25:17.054103: step 34520, loss 0.449222, acc 1
2019-05-02T21:25:17.200309: step 34530, loss 0.449113, acc 1
2019-05-02T21:25:17.346793: step 34540, loss 0.449166, acc 1
2019-05-02T21:25:17.492830: step 34550, loss 0.449071, acc 1
2019-05-02T21:25:17.638055: step 34560, loss 0.451502, acc 1
2019-05-02T21:25:17.786293: step 34570, loss 0.44912, acc 1
2019-05-02T21:25:17.931611: step 34580, loss 0.448916, acc 1
2019-05-02T21:25:18.077337: step 34590, loss 0.44892, acc 1
2019-05-02T21:25:18.223258: step 34600, loss 0.44889, acc 1

Evaluation:
2019-05-02T21:25:18.300525: step 34600, lo

2019-05-02T21:25:32.920691: step 35550, loss 0.44053, acc 1
2019-05-02T21:25:33.067726: step 35560, loss 0.440131, acc 1
2019-05-02T21:25:33.214344: step 35570, loss 0.441977, acc 1
2019-05-02T21:25:33.362382: step 35580, loss 0.440003, acc 1
2019-05-02T21:25:33.507915: step 35590, loss 0.440411, acc 1
2019-05-02T21:25:33.656111: step 35600, loss 0.441348, acc 1

Evaluation:
2019-05-02T21:25:33.732499: step 35600, loss 1.43015, acc 0.79
[UNOFFICIAL] (2*9+1)-Way Macro-Average F1 Score (excluding Other): 0.776432

2019-05-02T21:25:33.882909: step 35610, loss 0.439709, acc 1
2019-05-02T21:25:34.028135: step 35620, loss 0.442868, acc 1
2019-05-02T21:25:34.175167: step 35630, loss 0.440015, acc 1
2019-05-02T21:25:34.321221: step 35640, loss 0.445068, acc 1
2019-05-02T21:25:34.468793: step 35650, loss 0.4445, acc 1
2019-05-02T21:25:34.614496: step 35660, loss 0.439446, acc 1
2019-05-02T21:25:34.760961: step 35670, loss 0.439284, acc 1
2019-05-02T21:25:34.907024: step 35680, loss 0.439519, ac

## Tensorboard

```
tensorboard --logdir=./23.runs --host 0.0.0.0
```